In [1]:
import requests
import json
import time
import pandas as pd
from tqdm.auto import tqdm

place2_map = {
    1: "Tehran",
    2: "Karaj",
    3: "Mashhad",
    4: "Isfahan",
    5: "Tabriz",
    6: "Shiraz",
    7: "Ahvaz",
    8: "Ghom",
}

# https://api.divar.ir/v8/places/cities

In [2]:
def request_api(last_post_date=0):
    url = 'https://api.divar.ir/v8/web-search/1/apartment-sell'
    my_obj = {"json_schema":{"category":{"value":"apartment-sell"}, "cities":[str(i) for i in place2_map.keys()]}, "last-post-date":last_post_date}
    response = requests.post(url, json = my_obj)
    response_json = json.loads(response.text)
    return response_json

def flatten_dict(dd, separator='.', prefix=''):
    return {
        prefix + separator + k if prefix else k : v
        for kk, vv in dd.items()
        for k, v in flatten_dict(vv, separator, kk).items()
    } if isinstance(dd, dict) else { prefix : dd }

def retrieve_raw_dataset(iterations=60) -> pd.DataFrame:
    raw_dataset = []
    last_post_date=0
    for iteration in tqdm(range(iterations), desc="Iterations"):
        got_response = False
        while not got_response:
            try:
                response_json = request_api(last_post_date)
                for post_row in response_json["web_widgets"]['post_list']:
                    assert post_row['widget_type'] == "POST_ROW"
                    data = flatten_dict(post_row)
                    data["raw_response"] = response_json
                    raw_dataset.append(data)
                last_post_date = response_json["last_post_date"]  # Next Scroll
                time.sleep(1)
                got_response = True
            except Exception as e:
                print("API Call Failed!")
    return pd.DataFrame(raw_dataset)

In [102]:
df = retrieve_raw_dataset(iterations=1000)
df

Iterations:   0%|          | 0/1000 [00:00<?, ?it/s]

,widget_type,data.@type,data.image_url,data.image_count,data.title,data.has_chat,data.top_description_text,data.middle_description_text,data.bottom_description_text,data.red_text,...,data.image_top_left_tag.icon.icon_color,data.image_top_left_tag.text,data.image_top_left_tag.is_icon_left,data.image_overlay_tag.text,data.image_overlay_tag.image_url,data.image_overlay_tag.image_color,data.image_overlay_tag.icon.image_url_dark,data.image_overlay_tag.icon.image_url_light,data.image_overlay_tag.icon.icon_name,data.image_overlay_tag.icon.icon_color
0,POST_ROW,type.googleapis.com/widgets.PostRowData,,0,کلاهدوز صفر,True,,"۲,۸۵۰,۰۰۰,۰۰۰ تومان",آژانس املاک در مشهد,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,POST_ROW,type.googleapis.com/widgets.PostRowData,[{'src': 'https://s100.divarcdn.com/static/thu...,6,فروش ۱۰۰ متر آپارتمان ۲خواب // قیطریه جنوبی,False,,"۱۳,۲۰۰,۰۰۰,۰۰۰ تومان",آژانس املاک در تهران,,...,WHITE_PRIMARY,۶,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,POST_ROW,type.googleapis.com/widgets.PostRowData,[{'src': 'https://s100.divarcdn.com/static/thu...,16,آپارتمان ۱۱۰ متری کلید نخورده در خیابان مرتضوی,True,,"۵,۱۳۰,۰۰۰,۰۰۰ تومان",مشاور املاک در تهران,,...,WHITE_PRIMARY,۱۶,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,POST_ROW,type.googleapis.com/widgets.PostRowData,[{'src': 'https://s100.divarcdn.com/static/thu...,1,116متر ، نوساز ، فول امکانات / شهرک صدف,False,,"۲,۹۰۰,۰۰۰,۰۰۰ تومان",لحظاتی پیش در کرج,,...,WHITE_PRIMARY,۱,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,POST_ROW,type.googleapis.com/widgets.PostRowData,,0,قیطریه (کیکاووس) ۱۵۰ متر فول بازسازی,True,,"۱۲,۶۰۰,۰۰۰,۰۰۰ تومان",آژانس املاک در تهران,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,POST_ROW,type.googleapis.com/widgets.PostRowData,[{'src': 'https://s100.divarcdn.com/static/thu...,1,141متر/تاپ لوکیشن/دو پارکینگ/تک بازدید,False,,"۴,۰۵۰,۰۰۰,۰۰۰ تومان",آژانس املاک در مشهد,,...,WHITE_PRIMARY,۱,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23996,POST_ROW,type.googleapis.com/widgets.PostRowData,[{'src': 'https://s100.divarcdn.com/static/thu...,1,آپارتمان 100متری صفرتاصد بازسازی نزدیک اصلی,False,,"۱,۳۹۰,۰۰۰,۰۰۰ تومان",آژانس املاک در اهواز,,...,WHITE_PRIMARY,۱,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23997,POST_ROW,type.googleapis.com/widgets.PostRowData,,0,آپارتمان ۱۶۰ متر مهرشهر,False,,"۲,۸۰۰,۰۰۰,۰۰۰ تومان",۲۳ ساعت پیش در اهواز,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23998,POST_ROW,type.googleapis.com/widgets.PostRowData,,0,آپارتمان ۱30 متری الهیه,True,,"۳,۰۰۰,۰۰۰,۰۰۰ تومان",۲۳ ساعت پیش در مشهد,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [106]:
# Check for duplicates
assert df.duplicated(subset="data.action.payload.token").sum() == 0

In [9]:
df = df.drop(columns=["raw_response"])

In [11]:
df.to_csv("divar_apartment_sell.csv")